**Drive'a Bağlanma**

In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


**Veri Dosyalarını Yükleme**

In [2]:
import numpy as np

# Veri kümelerinin yolları
data_path = "/content/drive/MyDrive/Yazgel_Muzik/BolunmusVeriler"

# Eğitim, validasyon ve test veri kümelerini yükleme
train_data = np.load(f"{data_path}/train_data.npy", allow_pickle=True)
train_labels = np.load(f"{data_path}/train_labels.npy", allow_pickle=True)
val_data = np.load(f"{data_path}/val_data.npy", allow_pickle=True)
val_labels = np.load(f"{data_path}/val_labels.npy", allow_pickle=True)
test_data = np.load(f"{data_path}/test_data.npy", allow_pickle=True)
test_labels = np.load(f"{data_path}/test_labels.npy", allow_pickle=True)

# Veri boyutlarını kontrol etme
print(f"Eğitim kümesi boyutu: {len(train_data)}")
print(f"Validasyon kümesi boyutu: {len(val_data)}")
print(f"Test kümesi boyutu: {len(test_data)}")


Eğitim kümesi boyutu: 19416
Validasyon kümesi boyutu: 4854
Test kümesi boyutu: 2697


**Gerekli Kütüphaneler ve Model Yükleme**

In [3]:
!pip install transformers datasets torchaudio numpy

from transformers import Wav2Vec2Processor, Wav2Vec2ForSequenceClassification
import torchaudio
import os
from datasets import Dataset


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 33.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 22.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [4]:
pip install fsspec==2024.9.0


**Veri Yükleme ve Etiketleme**

In [5]:
import os

# Ses dosyalarının klasörleri
wav_folders = {
    "huzunlu": "/content/drive/MyDrive/Yazgel_Muzik/huzunlu",
    "agresif": "/content/drive/MyDrive/Yazgel_Muzik/agresif",
    "neseli": "/content/drive/MyDrive/Yazgel_Muzik/neseli"
}

data = []
labels = []

# WAV dosyalarını ve etiketleri yükleme
for label, folder in wav_folders.items():
    for file in os.listdir(folder):
        if file.endswith(".wav"):
            file_path = os.path.join(folder, file)
            data.append(file_path)
            labels.append(label)

print(f"Toplam veri: {len(data)}")
print(f"Etiketler: {set(labels)}")



Toplam veri: 27142
Etiketler: {'neseli', 'agresif', 'huzunlu'}


In [6]:
from sklearn.model_selection import train_test_split

# Eğitim, validasyon ve test kümelerine ayırma
train_data, test_data, train_labels, test_labels = train_test_split(
    data, labels, test_size=0.1, random_state=42, stratify=labels
)

train_data, val_data, train_labels, val_labels = train_test_split(
    train_data, train_labels, test_size=0.2, random_state=42, stratify=train_labels
)

print(f"Eğitim verisi: {len(train_data)}")
print(f"Validasyon verisi: {len(val_data)}")
print(f"Test verisi: {len(test_data)}")


Eğitim verisi: 19541
Validasyon verisi: 4886
Test verisi: 2715


In [7]:
from datasets import Dataset

# Dataset formatına dönüştürme
train_dataset = Dataset.from_dict({"path": train_data, "label": train_labels})
val_dataset = Dataset.from_dict({"path": val_data, "label": val_labels})
test_dataset = Dataset.from_dict({"path": test_data, "label": test_labels})

print("Dataset oluşturuldu.")


Dataset oluşturuldu.


**Model Eğitimi Başlatma**

In [8]:
from transformers import Wav2Vec2ForSequenceClassification, Trainer, TrainingArguments
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, roc_auc_score, confusion_matrix
import numpy as np
import torch
from transformers import Wav2Vec2Processor
import torchaudio

# Etiketler için bir harita oluşturuyoruz
label_map = {"neseli": 0, "huzunlu": 1, "agresif": 2}

# Wav2Vec işlemcisini yükleyelim
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base")

# Preprocessing fonksiyonu
def preprocess_function(batch):
    # Ses dosyalarını yükleyip sadece ilk kanalını alıyoruz (monofonik)
    audio = [torchaudio.load(path)[0][0].numpy() for path in batch["path"]]

    # Etiketleri sayısal verilere dönüştürüyoruz
    batch["label"] = [label_map[label] for label in batch["label"]]

    # Wav2Vec işlemcisi ile ses verilerini işliyoruz, padding=True ile uzunlukları eşitliyoruz
    inputs = processor(audio, sampling_rate=16000, return_tensors="pt", padding=True, truncation=True, max_length=240000)

    # İşlenmiş verileri batch'e ekleyelim
    batch["input_values"] = inputs.input_values.detach().numpy()  # PyTorch tensor'ü NumPy array'e çeviriyoruz
    batch["labels"] = batch["label"]  # Sayısal etiketleri batch'e ekliyoruz

    return batch

# Preprocessing'i uygulama (batched=True ile)
train_dataset = train_dataset.map(preprocess_function, batched=True)
val_dataset = val_dataset.map(preprocess_function, batched=True)
test_dataset = test_dataset.map(preprocess_function, batched=True)

print("Preprocessing tamamlandı.")

# Modeli yükleyelim
model = Wav2Vec2ForSequenceClassification.from_pretrained(
    "facebook/wav2vec2-base",
    num_labels=len(set(label_map))  # Etiket sayısını belirtiyoruz
)

# Eğitim parametreleri
training_args = TrainingArguments(
    output_dir="./results",          # Çıktı dosyalarının kaydedileceği klasör
    evaluation_strategy="epoch",     # Her epoch sonunda değerlendirme yap
    learning_rate=2e-5,              # Öğrenme oranı
    per_device_train_batch_size=16,  # Batch boyutu
    num_train_epochs=5,              # Epoch sayısı
    save_steps=500,                  # Model kaydetme sıklığı
    logging_dir="./logs",            # Log dosyalarının kaydedileceği klasör
    logging_steps=100                # Loglama sıklığı
)

# Trainer'ı hazırlayalım
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=processor
)

# Eğitimi başlatıyoruz
trainer.train()

# Modeli ve işlemciyi kaydedelim
model.save_pretrained("./final_model")
processor.save_pretrained("./final_model")

print("Model başarıyla kaydedildi.")

# Eğitim ve test kümesi üzerinde tahminler yapalım
predictions = trainer.predict(test_dataset)
pred_labels = predictions.predictions.argmax(axis=-1)  # Etiketleri tahmin ediyoruz

# Gerçek etiketlerle karşılaştırma yaparak doğruluğu hesaplıyoruz
test_labels_int = [label_map[label] for label in test_labels]

# Sınıflandırma metriklerini hesaplıyoruz
accuracy = accuracy_score(test_labels_int, pred_labels)
precision, recall, fscore, support = precision_recall_fscore_support(test_labels_int, pred_labels, average='weighted')
conf_matrix = confusion_matrix(test_labels_int, pred_labels)

# AUC skoru hesaplıyoruz (sadece binary sınıflandırma için)
try:
    auc = roc_auc_score(test_labels_int, predictions.predictions, multi_class='ovr', average='weighted')
except ValueError:
    auc = "AUC hesaplanamadı (muhtemelen binary sınıflandırma)."

# Sensitivity ve Specificity hesaplaması (binary sınıflandırma için)
sensitivity = recall
specificity = conf_matrix[1,1] / (conf_matrix[1,0] + conf_matrix[1,1]) if conf_matrix[1,0] + conf_matrix[1,1] != 0 else "Hesaplanamadı"

# Sonuçları yazdıralım
print(f"Test Doğruluğu: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F-Score: {fscore}")
print(f"AUC: {auc}")
print(f"Sensitivity: {sensitivity}")
print(f"Specificity: {specificity}")
print(f"Confusion Matrix:\n{conf_matrix}")

# Test sonuçlarını kaydedelim
with open("test_results.txt", "w") as f:
    f.write(f"Test Accuracy: {accuracy}\n")
    f.write(f"Precision: {precision}\n")
    f.write(f"Recall: {recall}\n")
    f.write(f"F-Score: {fscore}\n")
    f.write(f"AUC: {auc}\n")
    f.write(f"Sensitivity: {sensitivity}\n")
    f.write(f"Specificity: {specificity}\n")
    f.write(f"Confusion Matrix:\n{conf_matrix}")
    print("Sonuçlar başarıyla kaydedildi.")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/159 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/163 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.84k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/configuration_utils.py:311: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


...

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss
1,0.658800,0.606841
2,0.580800,0.544326
3,0.477600,0.508593
4,0.359800,0.473333
5,0.310200,0.491536


Model başarıyla kaydedildi.


Test Doğruluğu: 0.8261510128913444
Precision: 0.8265334879492569
Recall: 0.8261510128913444
F-Score: 0.8263037678819858
AUC: AUC hesaplanamadı (muhtemelen binary sınıflandırma).
Sensitivity: 0.8261510128913444
Specificity: 0.8982658959537573
Confusion Matrix:
[[731  81 107]
 [ 88 777  38]
 [117  41 735]]
Sonuçlar başarıyla kaydedildi.
